<a href="https://colab.research.google.com/github/raymondj80/CS50_Insulin_Calculator/blob/master/Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#TODO: Import Code
from google.colab import drive

#copies char between delim in infile
def copy_between(infile,delim):
  for line in infile:
      if delim in line.strip():
        s = line.strip()
        return s.strip(delim).strip('</')

# returns tth xrdml file as dictionary
def upload_xrdml(filename):
  # read tth and intensity values
  with open(filename,'r') as file:
    tth_beg = float(copy_between(file,'<startPosition>'))
    tth_end = float(copy_between(file,'<endPosition>'))
    intensities = [float(item) for item in copy_between(file,'<intensities unit="counts">').split(" ")]
    
  #create dictionary
    d = {}
    tth = tth_beg
    incr = (tth_end - tth_beg) / (len(intensities) - 1)
    for i in intensities:
      d[tth] = i
      tth = tth + incr
  return d

test = upload_xrdml('/content/drive/My Drive/Mundy Group/Three Muskasteers Group/Code/test.xrdml')



#TODO: Data Analysis



#TODO: Data Output